### Dairo Alberto Cuervo Garcia
#### Proyecto de grado - Exploración de discurso en cartas de maximos responsables - Maestría en Ciencias de los Datos y Analítica - Universidad EAFIT - 2024/1

## Preparación de texto y preprocesamiento

In [1]:
# Cargar librerias necesarias
import pandas as pd
import numpy as np
import re
import string
import nltk
import spacy
from nltk.tokenize import casual_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
from spacy.lang.es import Spanish
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Dependencias de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DairoAlbertoCuervoGa\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DairoAlbertoCuervoGa\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\DairoAlbertoCuervoGa\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DairoAlbertoCuervoGa\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
ruta_archivo_csv = r'C:\Users\DairoAlbertoCuervoGa\Downloads\Text_project\1.1.Output\Resultados.csv' ## Cargar archivo csv
df = pd.read_csv(ruta_archivo_csv)

### Limpiar datos
Se realiza una serie de pasos dentro de una función para limpiar texto antes de preprocesamiento

In [3]:
# Crear función de limpieza de texto
def limpiar_texto(texto):
    texto = re.sub(r'-', ' ', texto) # Reemplazar guiones por espacios vacíos
    texto = texto.lower() # Convertir a minúsculas
    texto = re.sub(r'\d+', '', texto) # Eliminar números y puntuaciones
    texto = texto.translate(str.maketrans("", "", string.punctuation))
    texto = re.sub(r'[^\w\s]', '', texto) # Eliminar caracteres especiales
    texto = re.sub(r'\s+', ' ', texto) # Eliminar espacios adicionales
    texto = re.sub(r'[\n\r]', ' ', texto) #Eliminar los saltos de línea (\n) y retornos de carro (\r) con la expresión regular r'[\n\r]'.
    texto = re.sub(r'\$\d+\.?\d*', '', texto) #Eliminar los símbolos de moneda con la expresión regular r'\$\d+\.?\d*' (para el caso de dólares). 
    texto = re.sub(r'\b([MDCLXVI]+)\b', '', texto) #Eliminar los números romanos con la expresión regular r'\b([MDCLXVI]+)\b'.
    texto = re.sub(r'\b\w\b', '', texto) #Eliminar las palabras que contienen solo una letra (como "a", "e", "o") con la expresión regular r'\b\w\b'.
    texto = reemplazar_comillas(texto)  #agregar función de reemplazar comillas dobles.
    return texto

# Función para reemplazar las comillas dobles por nada.
def reemplazar_comillas(texto):
    texto = texto.replace('"', '')
    return texto

df['texto_limpio'] = df['Texto'].apply(limpiar_texto)


### Tokenización, lematización y remoción de stopwords

#### Creación de listado de palabras a eliminar
- Se crea un listado de palabras a eliminar que limpia los textos procesados para la mejora de modelos, toma palabras de baja relevancia y con alta frecuencia

In [4]:
palabras_eliminar = ['millón','millon','carta','presidente','año','pais','billón', 'colombia', 'país','inmediatamente','anterior', 'él','el','grupo nutresa',
                     'grupo éxito','éxito','grupo', 'grupo exito','bavaria','grupo aval',
                     'bancolombia','corficolombiana','banco bogota', 'banco bogotá','banco de bogota', 
                     'banco de bogotá','davivienda','cementos argos', 'cemex', 'grupo argos','ecopetrol',
                     'isa','canacol','frontera energy','drummond','cerrejon','reficar','cenit','celsia',
                     'grupo de energia de bogotá', 'grupo de energia de bogota','grupo energia bogotá', 
                     'grupo energia bogota','sura','epm','claro','avianca','aval','bolívar','banco','bancos','daviplata','nutresa','argos'] 
nlp = spacy.load('es_core_news_sm') #palabras a eliminar
nlp.max_length = 1500000


##### Tokenización

In [5]:
stop_words_nltk = set(stopwords.words('spanish'))  # cargar stopwords en nltk

In [6]:
# Función que tokenize los textos
def tokenizar(text):
    tokens = casual_tokenize(str(text)) #retirar cambio de guiones por espacios vacíos y agregarlo en la preparación
    #tokens = nltk.tokenize.casual_tokenize(str(text))
    tokens = [w.lower() for w in tokens if len(w) > 3] #excluir tokens iguales o inferiores a 3 letras
    #tokens = [unidecode(w) for w in tokens] # activar para quitar tildes (no recomendado se deja comentado por que afecta los resultados)
    tokens = [w for w in tokens if w not in stop_words_nltk]
    tokens = [w for w in tokens if w.isalpha()]
    tokens = [w for w in tokens if w.isalpha() and w not in palabras_eliminar] #se agrega eliminar palabras especificas
    return " ".join(tokens)

df['texto_tokenizado'] = df['texto_limpio'].apply(tokenizar)

##### Lematización y remoción de stopwords
- Remoción de stopwords usando NLTK en español
- Lematización

In [7]:
# Crear función de lematizado y remoción de stop words
def lematizar_texto(texto):
    doc = nlp(texto)
    tokens_sin_stopwords = [token.lemma_ for token in doc if not token.is_stop and token.lemma_ not in palabras_eliminar] #agregar eliminar palabras especificas
    return " ".join(tokens_sin_stopwords)

df['texto_lematizado'] = df['texto_tokenizado'].apply(lematizar_texto)

#### Anexar a consulta y almacenar

In [8]:
# Agregar la columna de empresa como índice del dataframe
df['Empresa_Año'] = df['Empresa'] + '_' + df['Año'].astype(str)
df.set_index('Empresa_Año', inplace=True)

# Visualizar los primeros registros del DataFrame
#df.head()

In [9]:
# Guardar el DataFrame procesado en un nuevo archivo CSV
ruta_salida_csv = r'C:\Users\DairoAlbertoCuervoGa\Downloads\Text_project\1.1.Output\Resultados.csv'
df.to_csv(ruta_salida_csv)

# Visualizar los primeros registros del DataFrame
#df.head()

## Cálculo de número de condición
Este número ayuda a detectar la sensibilidad de las operaciones matem ́aticas frente a errores de redondeo,
se observa que a medida que se mejora la limpieza de los textos, el n umerode condición disminuye, este apartado tiene el resultado final, en el Anexo_escenarios_iniciales se recrea brevemente los resultados sin los pasos de procesamiento adecuado.

#### Creación de matriz TF-IDF
Indexación y creación de matriz de similitud por métrica del coseno

In [10]:
# Vectorización TF-IDF
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=2, norm='l2', encoding='utf-8', ngram_range=(1, 1)) #solo se procesan ngram=1
tf_idf_matrix = vectorizer.fit_transform(df['texto_lematizado'])

In [11]:
# Crear matriz de similitud
similarity_matrix = pd.DataFrame(cosine_similarity(tf_idf_matrix), index=df.index, columns=df.index)

In [12]:
# Calcular el número de condición de la matriz de similitud
cond_A = np.linalg.cond(similarity_matrix)

# Imprimir el resultado
print("El número de condición de la matriz de similitud es:", cond_A)

El número de condición de la matriz de similitud es: 85.77500800225008
